## 요약(Summary) Evaluators

일부 메트릭은 실험의 개별 실행이 아닌 전체 실험 수준에서만 정의할 수 있습니다. 

예를 들어, 데이터 세트에서 시작된 실험의 **모든 실행에 걸쳐 분류자의 평가 점수를 계산** 하고 싶을 수 있습니다. 

이를 `summary_evaluators` 라고 합니다. 

이러한 평가자는 하나의 Run과 Example 대신 각각의 목록을 가져옵니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [ ]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question

GPT-4o-mini 모델과 Ollama 모델을 활용하여 질문에 대한 답변을 생성하는 함수를 생성합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
ollama_chain = ask_question_with_llm(ChatOllama(model="EEVE-Korean-10.8B:latest"))

`OpenAIRelevanceGrader` 는 질문(Question), 컨텍스트(Context), 답변(Answer) 가 관련성이 있는지 여부를 평가하는 데 사용됩니다.

- `target="retrieval-question"`: 질문과 컨텍스트가 관련성이 있는지 여부를 평가합니다.
- `target="retrieval-answer"`: 답변과 컨텍스트가 관련성이 있는지 여부를 평가합니다.

In [ ]:
from langchain_teddynote.evaluator import OpenAIRelevanceGrader
from langchain_openai import ChatOpenAI


rq_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-question"
).create()

ra_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-answer"
).create()

In [ ]:
rq_grader.invoke(
    {
        "input": "삼성전자가 자체 개발한 생성형 AI 의 이름은?",
        "context": "삼성전자 AI 는 빅스비에요",
    }
)

In [ ]:
ra_grader.invoke(
    {
        "input": "삼성전자가 자체 개발한 생성형 AI 는 가우스 입니다.",
        "context": "삼성전자 AI 는 빅스비에요",
    }
)

## 관련성(Relevance) 평가를 종합하는 Summary Evaluator

In [ ]:
from typing import List
from langsmith.schemas import Example, Run
from langsmith.evaluation import evaluate


def relevance_score_summary_evaluator(runs: List[Run], examples: List[Example]) -> dict:
    rq_scores = 0  # 질문 관련성 점수
    ra_scores = 0  # 답변 관련성 점수

    for run, example in zip(runs, examples):
        question = example.inputs["question"]
        context = run.outputs["context"]
        prediction = run.outputs["answer"]

        # 질문 관련성 평가
        rq_score = rq_grader.invoke(
            {
                "input": question,
                "context": context,
            }
        )
        # 답변 관련성 평가
        ra_score = ra_grader.invoke(
            {
                "input": prediction,
                "context": context,
            }
        )

        # 관련성 점수 누적
        if rq_score.score == "yes":
            rq_scores += 1
        if ra_score.score == "yes":
            ra_scores += 1

    # 최종 관련성 점수 계산 (질문 관련성과 답변 관련성의 평균)
    final_score = ((rq_scores / len(runs)) + (ra_scores / len(runs))) / 2

    return {"key": "relevance_score", "score": final_score}

평가를 진행합니다.

In [ ]:
# 평가 실행
dataset_name = "RAG_EVAL_DATASET"

experiment_result1 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o-mini 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

# 평가 실행
experiment_result2 = evaluate(
    ollama_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Ollama(EEVE-Korean-10.8B:latest) 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

결과를 확인합니다. 

(참고) 개별 데이터셋에 대한 평가는 확인할 수 없으며, 실험(Experiment) 단위로 확인할 수 있습니다.

![](./assets/eval-08.png)